### Imports

In [1]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
import numpy as np
import time
import struct
import random

from scipy.signal import find_peaks

In [2]:
project_name = "protected0.5"
num_traces = 10000
scmd_value = 1 #0 for unprotected and 1 for protected

In [5]:
project_name = "unprotected"
num_traces = 2000
scmd_value = 0 #0 for unprotected and 1 for protected

In [5]:
min_in_val = -2
max_in_val = 2
decimate_value = 1
filename = project_name + "-trace.txt"

### Function Definitions

In [8]:
def random_float(min_val, max_val):
    # Generate a random float between min_val and max_val
    rand_float = random.uniform(min_val, max_val)
    # Round to 2 decimal places
    return round(rand_float, 2)

In [10]:
def float_to_bytearray_32bit_little_edian(f):
    # Pack the float as a 32-bit (4-byte) IEEE 754 floating point number
    packed = struct.pack('f', f)
    # Convert to bytearray
    return bytearray(packed)

In [12]:
def scope_setup(samples=24431, decimate=2):
    # arm the scope
    
    scope.arm()
    
    # Set the maximum number of points in a trace
    scope.adc.fifo_fill_mode = "normal"
    scope.adc.samples = samples
    scope.adc.decimate = decimate

In [14]:
def capture_trace(cmd_data, cmd='p', scmd=scmd_value, prints=True):
    scope.arm()
    # flush the UART buffer
    target.flush()
    
    target.send_cmd(cmd, scmd, cmd_data)
    ret = scope.capture()
    trace = scope.get_last_trace()
    
    returned_data = target.read_cmd('r')
    ack = target.read_cmd('e')
    if prints:
        print(f'r\t- target.read_cmd("r"):\t{returned_data}')
        print(f'ack\t- target.read_cmd("e"):\t{ack}')
    return trace
    

### Target Setup

In [19]:
pip install libusb1


Note: you may need to restart the kernel to use updated packages.


In [17]:
#Scope setup
scope = cw.scope()
scope.default_setup()

target = cw.target(scope,cw.targets.SimpleSerial2) #cw.targets.SimpleSerial can be omitted
#MY CHANGES - changed target to SimpleSerial2 - to be able to send_cmd

(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.64.0) is outdated - latest is 0.65.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 1920699                   to 24823583                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 29538471                 
scope.clock.adc_rate                     changed from 0.0                       to 29538471.0               
scope.clock.clkgen_

In [19]:
scope_setup(samples=24430, decimate=decimate_value)

In [21]:
%%bash
cd network/
make PLATFORM='CWLITEARM' CRYPTO_TARGET=NONE 

SS_VER set to SS_VER_2_1
rm -f -- simpleserial-target-CWLITEARM.hex
rm -f -- simpleserial-target-CWLITEARM.eep
rm -f -- simpleserial-target-CWLITEARM.cof
rm -f -- simpleserial-target-CWLITEARM.elf
rm -f -- simpleserial-target-CWLITEARM.map
rm -f -- simpleserial-target-CWLITEARM.sym
rm -f -- simpleserial-target-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- main.s network.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- main.d network.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- main.i network.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10.3-2021.10) 10.3.1 20210824 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In file included from main.c:15:
network_config.h:14:6: warning: function declaration isn't a prototype [-Wstrict-prototypes]
   14 | void init_weights() {
      |      ^~~~~~~~~~~~


.
Compiling C: network.c
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -gdwarf-2 -DSS_VER=SS_VER_2_1 -DSTM32F303xC -DSTM32F3 -DSTM32 -DDEBUG -DHAL_TYPE=HAL_stm32f3 -DPLATFORM=CWLITEARM -DF_CPU=7372800UL -DSS_VER_2_0=2 -DSS_VER_1_1=1 -DSS_VER_1_0=0 -Os -funsigned-char -funsigned-bitfields -fshort-enums -Wall -Wstrict-prototypes -Wa,-adhlns=objdir/network.lst -I./simpleserial/ -I./hal -I./hal/stm32f3 -I./hal/stm32f3/CMSIS -I./hal/stm32f3/CMSIS/core -I./hal/stm32f3/CMSIS/device -I./hal/stm32f4/Legacy -std=gnu99  -MMD -MP -MF .dep/network.o.d network.c -o objdir/network.o
.
Compiling C: ./simpleserial/simpleserial.c
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -gdwarf-2 -DSS_VER=SS_VER_2_1 -DSTM32F303xC -DSTM32F3 -DSTM32 -DDEBUG -DHAL_TYPE=HAL_stm32f3 -DPLATFORM=CWLITEARM -DF_CPU=7372800UL -DSS_VER_2_0=2 -DSS_VER_1_1=1 -DSS_VER_1_0=0 -Os -funsigned-char -funsigned-bitfield

In [23]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "network/simpleserial-target-CWLITEARM.hex")

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 19107 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 19107 bytes


### Initialize the project

The Chipwhisperer `Project` class can be used to keep a collection of traces. 

In [25]:
proj = cw.create_project(project_name)

In [26]:


input_vals = [[0.5]*7 for _ in range(num_traces)]

for i in range(num_traces):
    if i % 7 == 0:
        input_vals[i][0] = random.uniform(-2, 2)

first_col = [row[0] for row in input_vals]
print("rows:", len(input_vals), "cols:", len(input_vals[0]))
print("max(first_col) =", max(first_col))
print("min(first_col) =", min(first_col))

rows: 10000 cols: 7
max(first_col) = 1.9998042265991436
min(first_col) = -1.9992547609954174


### Trace collection

In [28]:
float_val = -0.657
float_bytearray = float_to_bytearray_32bit_little_edian(float_val)
data = bytearray([0x42] * 4)
for i in range(50):
    trace_wave = capture_trace(float_bytearray, scmd=scmd_value)
print("warm up done")

start = time.time() 
completed_counter = 0 

for i in range (num_traces): 
    if isinstance(input_vals, np.ndarray):
        first_val = input_vals[i, 0]
    else:
        first_val = input_vals[i][0]

    cmd_data = float_to_bytearray_32bit_little_edian(first_val)
    
    trace_wave = capture_trace(cmd_data=cmd_data, scmd=scmd_value, prints=False)
    trace      = cw.Trace(wave=trace_wave,
                          textin=first_val,  
                          textout=None,
                          key=None)
    proj.traces.append(trace)
    
    completed_counter += 1
    if completed_counter % 100 == 0:
        print(f'completed {completed_counter} traces in\t{time.time() - start:.2f} seconds')

end = time.time()
print(f'capturing traces finished in {end - start:.2f} seconds!')

r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.read_cmd("e"):	CWbytearray(b'00 65 01 00 eb 00')
r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.read_cmd("e"):	CWbytearray(b'00 65 01 00 eb 00')
r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.read_cmd("e"):	CWbytearray(b'00 65 01 00 eb 00')
r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.read_cmd("e"):	CWbytearray(b'00 65 01 00 eb 00')
r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.read_cmd("e"):	CWbytearray(b'00 65 01 00 eb 00')
r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.read_cmd("e"):	CWbytearray(b'00 65 01 00 eb 00')
r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.read_cmd("e"):	CWbytearray(b'00 65 01 00 eb 00')
r	- target.read_cmd("r"):	CWbytearray(b'00 72 04 27 31 28 bf 63 00')
ack	- target.r

In [33]:
# proj.save()
# proj.close()

### Plot trace

In [36]:
proj = cw.open_project(project_name)

OSError: File /Users/andrew/Desktop/protectedvsunprotected/only-traces/capture_traces/protected0.5.cwp does not exist or is not a file

In [38]:
print(len(proj.traces))

10000


In [40]:
trace_waves_arr = []
for trace in proj.traces:
    trace_waves_arr.append(trace.wave)

In [39]:
import holoviews as hv
hv.extension('bokeh')
list_of_curves = [
    hv.Curve(trace_waves_arr[0], label='one trace'), 
]
hv.Overlay(list_of_curves).opts(
    height=600, 
    width=800
)

ModuleNotFoundError: No module named 'holoviews'

In [42]:
def disconnect_DUT():
    scope.dis()
    target.dis()
    return
disconnect_DUT()

In [ ]:
# ## save one trace to file
# f = open(filename, "w")
# i = 0
# f.write("x y\n")
# for point in trace_waves_arr[0]:
#     f.write(str(i)+" "+str(point))
#     f.write("\n")
#     i = i+1
# f.close()

### Save traces as txt files

In [44]:
import os
import numpy as np



def save_files(folder, trace_array, input_file, input_array):
    os.makedirs(folder, exist_ok=True)
    
    N = trace_array.shape[0]
    for n in range(N):
        trace_path = os.path.join(folder, f"trace_{n}.txt")
        with open(trace_path, "w") as f:
            for sample in trace_array[n]:
                f.write(f"{sample}\n")
    
    inputs_path = os.path.join(folder, input_file)
    with open(inputs_path, "w") as f:
        for row in input_array:
            line = " ".join(f"{x:.8f}" for x in row)
            f.write(line + "\n")

trace_waves_arr = np.array(trace_waves_arr)   
inputs_arr      = np.array(input_vals)       

save_files(
    folder      = project_name,
    trace_array = trace_waves_arr,
    input_file  = "inputs.txt",
    input_array = inputs_arr
)


In [ ]:
proj = cw.open_project(project_name)

In [46]:
trace_waves_arr = []
inputs_arr = []
for trace in proj.traces:
    trace_waves_arr.append(trace.wave)
    inputs_arr.append(trace.textin)

trace_waves_arr = np.array(trace_waves_arr)
print(len(trace_waves_arr))
print(len(inputs_arr))

2000
2000


In [50]:
import os

def save_files(folder, array, input_file, input_array):
    isExist = os.path.exists(folder)
    if not isExist:
        os.makedirs(folder)
    no_of_traces = len(input_array)
    for n in range(no_of_traces):
        with open(folder + "/trace_"+str(n)+".txt","w+") as file:
            for record in array[n]:
                file.write(str(record)+"\n")
        file.close()
    
    with open(folder + "/" + input_file,"w+") as file:
        for i in range(no_of_traces):
            file.write(str(input_array[i])+"\n")
        file.close()
    return

In [41]:
print(len(trace_waves_arr[0]))

5000


In [52]:
save_files(project_name, trace_waves_arr, "inputs.txt", inputs_arr)

### Zip files

In [ ]:
import shutil
shutil.make_archive(project_name, 'zip', project_name)
# shutil.make_archive(output_filename_dont_add_.zip, 'zip', directory_to_download)

### Delete files

In [5]:
import shutil

shutil.rmtree('5')

### Unzip file

In [ ]:
# import zipfile as zf
# files = zf.ZipFile("version_02.zip", 'r')
# files.extractall('network')
# files.close()